In [1]:
import pandas as pd

df = pd.read_csv("subset_dataset_cs2cd.csv")

C:\Users\kilua\AppData\Local\Temp\ipykernel_29936\920301165.py:3: DtypeWarning: Columns (9,22,31,33,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("subset_dataset_cs2cd.csv")


In [3]:
df.shape

(9437218, 42)

In [8]:
df.columns

Index(['X', 'Y', 'Z', 'tick', 'steamid', 'velocity_X', 'velocity_Y',
       'velocity_Z', 'is_airborne', 'is_walking', 'yaw', 'pitch',
       'usercmd_mouse_dx', 'usercmd_mouse_dy', 'usercmd_viewangle_x',
       'usercmd_viewangle_y', 'aim_punch_angle', 'i_recoil_idx',
       'fl_recoil_idx', 'active_weapon', 'active_weapon_ammo',
       'total_ammo_left', 'FIRE', 'last_shot_time', 'next_primary_attack_tick',
       'shots_fired', 'last_shot_time.1', 'kills_total', 'assists_total',
       'damage_total', 'headshot_kills_total', 'spotted',
       'approximate_spotted_by', 'is_scoped', 'health', 'armor_value',
       'is_alive', 'is_cheater', 'map', 'server', 'avg_rank',
       'match_making_type'],
      dtype='object')

In [ ]:
import pandas as pd
import numpy as np
import re
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)

print("--- PASO 1: Carga Eficiente de Datos ---")

filename = 'subset_dataset_cs2cd.csv'

columns_to_load = [
    'aim_punch_angle', 'spotted', 'approximate_spotted_by',
    'X', 'Y', 'Z', 'velocity_X', 'velocity_Y', 'velocity_Z', 'is_airborne',
    'is_walking', 'yaw', 'pitch', 'usercmd_mouse_dx', 'usercmd_mouse_dy',
    'usercmd_viewangle_x', 'usercmd_viewangle_y', 'i_recoil_idx',
    'fl_recoil_idx', 'active_weapon_ammo', 'total_ammo_left',
    'FIRE', 'last_shot_time', 'next_primary_attack_tick', 'shots_fired',
    'kills_total', 'assists_total', 'damage_total', 'headshot_kills_total',
    'is_scoped', 'health', 'armor_value', 'is_alive', 'avg_rank',
    'map', 'match_making_type',
    'is_cheater'
]

try:
    print(f"Cargando datos desde '{filename}'...")
    df = pd.read_csv(filename, usecols=lambda c: c in columns_to_load, low_memory=False)
    print("Datos cargados exitosamente.")
    print(f"Dimensiones del DataFrame: {df.shape}")
except FileNotFoundError:
    print(f"Error: No se encontró el archivo '{filename}'. Asegúrate de que el nombre y la ruta son correctos.")
    exit()
except Exception as e:
    print(f"Ocurrió un error al cargar los datos: {e}")
    exit()

print("\n--- PASO 2: Preprocesamiento y Optimización ---")

df_model = df

print("Limpiando y convirtiendo tipos de dato iniciales...")
for col in ['is_airborne', 'is_alive']:
    if col in df_model.columns:
        df_model[col] = df_model[col].astype(int)

for col in ['is_walking', 'is_scoped']:
    if col in df_model.columns:
        df_model.loc[:, col] = df_model[col].map({'True': 1, 'False': 0, True: 1, False: 0}).fillna(0)

print("Buscando y expandiendo columnas de vectores...")
vector_like_columns = [col for col in df_model.columns if df_model[col].dtype == 'object' and df_model[col].astype(str).str.contains(r'\[', regex=True, na=False).any()]

for col_name in vector_like_columns:
    print(f"Procesando y expandiendo '{col_name}'...")
    extracted = df_model[col_name].fillna('').astype(str).str.findall(r'(-?\d+\.?\d*e?-?\d+)')
    temp_df = pd.DataFrame(extracted.tolist(), index=df_model.index).astype(float)
    if not temp_df.empty:
        num_components = temp_df.shape[1]
        temp_df.columns = [f"{col_name}_{i}" for i in range(num_components)]
        df_model = df_model.join(temp_df)
    df_model.drop(columns=[col_name], inplace=True)

print("Codificando variables categóricas...")
categorical_features = [col for col in df_model.columns if df_model[col].dtype == 'object' or isinstance(df_model[col].dtype, pd.CategoricalDtype)]
if categorical_features:
    df_model = pd.get_dummies(df_model, columns=categorical_features, drop_first=True, dummy_na=True)

print("Rellenando valores nulos restantes...")
for col in df_model.select_dtypes(include=np.number).columns:
    if df_model[col].isnull().any():
        df_model[col] = df_model[col].fillna(df_model[col].median())

print("Reduciendo precisión de floats a float32...")
for col in df_model.select_dtypes(include=['float64']).columns:
    df_model[col] = df_model[col].astype('float32')

print("Preprocesamiento completado.")
print(f"Dimensiones finales del DataFrame para el modelo: {df_model.shape}")

print("\n--- PASO 3: División y Escalado de Datos ---")
X = df_model.drop(columns=['is_cheater'])
y = df_model['is_cheater']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=RANDOM_STATE, stratify=y
)

print("Escalando datos... (esto puede tardar debido al tamaño)")
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("\n--- PASO 4: Entrenamiento del Modelo Random Forest ---")
modelo_basico_rf = RandomForestClassifier(
    n_estimators=100,
    class_weight='balanced',
    random_state=RANDOM_STATE,
    n_jobs=-1,
    max_depth=20
)

start_time = time.time()
modelo_basico_rf.fit(X_train, y_train)
end_time = time.time()
print(f"Entrenamiento completado en {((end_time - start_time) / 60):.2f} minutos.")

print("\n--- PASO 5: Evaluación del Rendimiento ---")
predicciones = modelo_basico_rf.predict(X_test)
print("\n--- Reporte de Clasificación del Modelo Básico (Random Forest) ---")
print(classification_report(y_test, predicciones, target_names=['Legítimo (0)', 'Tramposo (1)']))
print("\n--- Matriz de Confusión ---")
print(confusion_matrix(y_test, predicciones))

In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

# Para asegurar la reproducibilidad
tf.random.set_seed(42)
np.random.seed(42)

# --------------------------------------------------------------------------
# PASO 1: CARGA EFICIENTE DE DATOS (Sin cambios)
# --------------------------------------------------------------------------
print("\nPaso 1: Definiendo columnas a cargar...")
columns_to_load = [
    'velocity', 'position', 'aim_punch_angle', 'usercmd_viewangle_x', 'usercmd_viewangle_y',
    'is_airborne', 'is_walking', 'yaw', 'pitch', 'usercmd_mouse_dx', 'usercmd_mouse_dy',
    'i_recoil_idx', 'fl_recoil_idx', 'active_weapon_ammo', 'total_ammo_left',
    'FIRE', 'last_shot_time', 'kills_total', 'assists_total', 'damage_total',
    'headshot_kills_total', 'spotted', 'approximate_spotted_by', 'is_scoped',
    'health', 'armor_value', 'is_alive', 'avg_rank',
    'active_weapon_name', 'map', 'match_making_type',
    'is_cheater'
]
try:
    print("Cargando datos...")
    df = pd.read_csv('subset_cs2cd.csv', usecols=lambda c: c in columns_to_load)
    print("Datos cargados exitosamente.")
except FileNotFoundError:
    print("Error: Reemplaza 'tu_archivo.csv' con el nombre de tu archivo de datos.")
    exit()

# --------------------------------------------------------------------------
# PASO 2: PREPROCESAMIENTO Y OPTIMIZACIÓN (CON CORRECCIONES)
# --------------------------------------------------------------------------
print("\nPaso 2: Preprocesando y optimizando...")
df_model = df

# --- A. Optimización de Tipos de Dato Categóricos ---
for col in df_model.select_dtypes(include=['object']).columns:
    if not df_model[col].astype(str).str.contains(r'\[', regex=True, na=False).any():
        df_model[col] = df_model[col].astype('category')

# --- B. Parseo y Expansión de Vectores ---
vector_like_columns = [col for col in df_model.columns if df_model[col].dtype == 'object']
for col_name in vector_like_columns:
    extracted = df_model[col_name].fillna('').astype(str).str.findall(r'(-?\d+\.?\d*e?-?\d+)')
    temp_df = pd.DataFrame(extracted.tolist(), index=df_model.index).astype(float)
    if not temp_df.empty:
        num_components = temp_df.shape[1]
        temp_df.columns = [f"{col_name}_{i}" for i in range(num_components)]
        df_model = df_model.join(temp_df)
    df_model.drop(columns=[col_name], inplace=True)

# --- C. Definición Final y Limpieza ---
target = 'is_cheater'
numerical_features = [col for col in df_model.columns if pd.api.types.is_numeric_dtype(df_model[col].dtype) and col != target]
categorical_features = [col for col in df_model.columns if isinstance(df_model[col].dtype, pd.CategoricalDtype)]

# Corregir el FutureWarning:
for col in numerical_features:
    if df_model[col].isnull().any():
        # Forma correcta y explícita de rellenar NaNs, sin 'inplace=True'
        df_model[col] = df_model[col].fillna(df_model[col].median())

if categorical_features:
    df_model = pd.get_dummies(df_model, columns=categorical_features, drop_first=True)

# --- D. Reducción de Precisión para Ahorrar Memoria (NUEVO PASO CLAVE) ---
print("Reduciendo precisión de floats a float32 para ahorrar memoria...")
for col in df_model.select_dtypes(include=['float64']).columns:
    df_model[col] = df_model[col].astype('float32')

# --- E. Separación y Escalado ---
X = df_model.drop(columns=[target])
y = df_model[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("Escalando datos... (esto puede tardar)")
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print("Preprocesamiento completado.")

# --------------------------------------------------------------------------
# PASO 3: CONSTRUCCIÓN Y ENTRENAMIENTO DEL MODELO (Sin cambios)
# --------------------------------------------------------------------------
print("\nPaso 3: Construyendo y entrenando el modelo...")
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001, verbose=1)
history = model.fit(X_train, y_train,
                    epochs=25,
                    batch_size=1024,
                    validation_split=0.2,
                    verbose=1,
                    callbacks=[reduce_lr])
print("Entrenamiento finalizado.")

# --------------------------------------------------------------------------
# PASO 4: EVALUACIÓN DEL MODELO (Sin cambios)
# --------------------------------------------------------------------------
print("\nPaso 4: Evaluando el rendimiento del modelo...")
loss, accuracy, precision, recall = model.evaluate(X_test, y_test, verbose=0)
predictions_prob = model.predict(X_test)
binary_predictions = (predictions_prob > 0.5).astype(int)
print(f'\nResultados en el conjunto de prueba:')
print(f'Pérdida (Loss): {loss:.4f}')
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, binary_predictions))
print("\nReporte de Clasificación:")
print(classification_report(y_test, binary_predictions))


Paso 1: Definiendo columnas a cargar...
Cargando datos...


C:\Users\kilua\AppData\Local\Temp\ipykernel_28488\2892000779.py:32: DtypeWarning: Columns (9,22,29,31,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('subset_cs2cd.csv', usecols=lambda c: c in columns_to_load)


Datos cargados exitosamente.

Paso 2: Preprocesando y optimizando...
Reduciendo precisión de floats a float32 para ahorrar memoria...
Escalando datos... (esto puede tardar)


MemoryError: Unable to allocate 12.8 GiB for an array with shape (70, 24573374) and data type float64

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

# Para asegurar la reproducibilidad de los resultados
tf.random.set_seed(42)
np.random.seed(42)

# --------------------------------------------------------------------------
# PASO 1: CARGA EFICIENTE DE DATOS
# --------------------------------------------------------------------------
print("\nPaso 1: Definiendo columnas a cargar y cargando datos eficientemente...")

# Define de antemano SÓLO las columnas que realmente necesitas del archivo CSV.
# Esto reduce drásticamente el uso de memoria al cargar.
# NOTA: Asegúrate de que los nombres de las columnas que contienen vectores
#       (como 'velocity', 'position', etc.) estén en esta lista.
columns_to_load = [
    # Columnas que sospechamos son vectores (ejemplos, ajusta a tu archivo)
    'velocity', 'position', 'aim_punch_angle',

    # Columnas numéricas directas
    'is_airborne', 'is_walking', 'yaw', 'pitch', 'usercmd_mouse_dx', 'usercmd_mouse_dy',
    'i_recoil_idx', 'fl_recoil_idx', 'active_weapon_ammo', 'total_ammo_left',
    'FIRE', 'last_shot_time', 'kills_total', 'assists_total', 'damage_total',
    'headshot_kills_total', 'spotted', 'approximate_spotted_by', 'is_scoped',
    'health', 'armor_value', 'is_alive', 'avg_rank',

    # Columnas categóricas
    'map', 'match_making_type',

    # La variable objetivo
    'is_cheater'
]

# Carga el CSV usando solo las columnas necesarias para ahorrar memoria
# Reemplaza 'tu_archivo.csv' con el nombre real de tu archivo
try:
    df = pd.read_csv('tu_archivo.csv', usecols=lambda c: c in columns_to_load)
    print("Datos cargados exitosamente.")
except FileNotFoundError:
    print("Error: Reemplaza 'tu_archivo.csv' con el nombre de tu archivo de datos.")
    # Salir si el archivo no se encuentra, o manejar como prefieras
    exit()

# --------------------------------------------------------------------------
# PASO 2: OPTIMIZACIÓN DE MEMORIA Y PREPROCESAMIENTO
# --------------------------------------------------------------------------
print("\nPaso 2: Optimizando tipos de datos y preprocesando...")

# NO HACEMOS COPIA: df_model ahora apunta al mismo objeto que df
# Esto es crucial para no duplicar el uso de memoria.
df_model = df

# --- A. Optimización de Tipos de Dato Categóricos ---
# Convertir columnas de texto a 'category' es una de las mayores ganancias de memoria.
for col in df_model.select_dtypes(include=['object']).columns:
    # Solo convertimos a categoría si no es una columna de vector que vamos a parsear
    if not df_model[col].str.contains(r'\[', regex=True, na=False).any():
        print(f"Convirtiendo '{col}' a tipo 'category'...")
        df_model[col] = df_model[col].astype('category')

# --- B. Parseo y Expansión de Vectores ---
print("Buscando y expandiendo columnas de vectores...")
vector_like_columns = [col for col in df_model.columns if df_model[col].dtype == 'object' and df_model[col].astype(str).str.contains(r'\[', regex=True, na=False).any()]

for col_name in vector_like_columns:
    # (El código para expandir vectores sigue igual que en la versión anterior)
    print(f"Procesando y expandiendo '{col_name}'...")
    extracted = df_model[col_name].fillna('').astype(str).str.findall(r'(-?\d+\.?\d*e?-?\d+)')
    temp_df = pd.DataFrame(extracted.tolist(), index=df_model.index).astype(float)
    if not temp_df.empty:
        num_components = temp_df.shape[1]
        temp_df.columns = [f"{col_name}_{i}" for i in range(num_components)]
        df_model = df_model.join(temp_df)
    df_model.drop(columns=[col_name], inplace=True)

# --- C. Definición Final de Features y Preprocesamiento ---
# (El resto del preprocesamiento es similar a la versión anterior)
target = 'is_cheater'
numerical_features = [col for col in df_model.columns if pd.api.types.is_numeric_dtype(df_model[col].dtype) and col != target]
categorical_features = [col for col in df_model.columns if isinstance(df_model[col].dtype, pd.CategoricalDtype)]

for col in numerical_features:
    if df_model[col].isnull().any():
        df_model[col].fillna(df_model[col].median(), inplace=True)

if categorical_features:
    df_model = pd.get_dummies(df_model, columns=categorical_features, drop_first=True)

X = df_model.drop(columns=[target])
y = df_model[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print("Preprocesamiento completado.")


# --------------------------------------------------------------------------
# PASO 3: PREPROCESAMIENTO FINAL
# --------------------------------------------------------------------------
print("\nPaso 3: Preprocesamiento final...")

# --- A. Manejo de Valores Faltantes ---
print("Manejando valores faltantes...")
for col in numerical_features:
    if df_model[col].isnull().any():
        median_val = df_model[col].median()
        df_model[col].fillna(median_val, inplace=True)

for col in categorical_features:
    if df_model[col].isnull().any():
        df_model[col].fillna('missing', inplace=True)

# --- B. One-Hot Encoding ---
if categorical_features:
    print("Aplicando One-Hot Encoding...")
    df_model = pd.get_dummies(df_model, columns=categorical_features, drop_first=True)

# --- C. Separación de Datos ---
print("Separando datos...")
X = df_model.drop(columns=[target])
y = df_model[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# --- D. Escalado ---
print("Escalando características...")
# Guardamos los nombres de las columnas para poder reconstruir el DataFrame más tarde si es necesario
X_train_cols = X_train.columns
X_test_cols = X_test.columns
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Preprocesamiento completado.")

# ... (El resto del código para construir, entrenar y evaluar el modelo sigue igual) ...

# --------------------------------------------------------------------------
# PASO 4: CONSTRUCCIÓN DEL MODELO DE DEEP LEARNING
# --------------------------------------------------------------------------
print("\nPaso 4: Construyendo el modelo de red neuronal...")
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')])
model.summary()

# --------------------------------------------------------------------------
# PASO 5: ENTRENAMIENTO DEL MODELO
# --------------------------------------------------------------------------
print("\nPaso 5: Entrenando el modelo...")
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001, verbose=1)
history = model.fit(X_train, y_train,
                    epochs=25,
                    batch_size=1024,
                    validation_split=0.2,
                    verbose=1,
                    callbacks=[reduce_lr])
print("Entrenamiento finalizado.")

# --------------------------------------------------------------------------
# PASO 6: EVALUACIÓN DEL MODELO
# --------------------------------------------------------------------------
print("\nPaso 6: Evaluando el rendimiento del modelo...")
loss, accuracy, precision, recall = model.evaluate(X_test, y_test, verbose=0)
predictions_prob = model.predict(X_test)
binary_predictions = (predictions_prob > 0.5).astype(int)
print(f'\nResultados en el conjunto de prueba:')
print(f'Pérdida (Loss): {loss:.4f}')
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, binary_predictions))
print("\nReporte de Clasificación:")
print(classification_report(y_test, binary_predictions))


Paso 1: Definiendo columnas a cargar y cargando datos eficientemente...
Error: Reemplaza 'tu_archivo.csv' con el nombre de tu archivo de datos.

Paso 2: Optimizando tipos de datos y preprocesando...
Convirtiendo 'steamid' a tipo 'category'...


AttributeError: Can only use .str accessor with string values!

: 